In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
               
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [48]:
fy_10 = pd.read_csv('fy_10unique.csv')
fy_11 = pd.read_csv('fy_11unique.csv')
fy_12 = pd.read_csv('fy_12unique.csv')
fy_13 = pd.read_csv('fy_13unique.csv')
fy_14 = pd.read_csv('fy_14unique.csv')
fy_15 = pd.read_csv('fy_15unique.csv')


In [49]:
fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15]

In [50]:
for index, year in enumerate(fy_list):
    fy_list[index].drop('key', axis=1, inplace=True)
    

In [217]:
dtypes_df = pd.DataFrame()
dtypes_df['dtype'] = fy_list[0].dtypes.index
for index, year in enumerate(fy_list):
    dtypes_df[index] = fy_list[index].dtypes.values
dtypes_df

In [101]:
group_by = ['Fund Id',
           'Fund Center Id', 'GL Account', 'GL Category',
           'GL Category Description',
           'Revenue or Expenditure']

essentials = ['Current Budget',
       'Fiscal Year', 'Fund Center Id', 'Fund Id','GL Account', 'GL Category',
       'GL Category Description', 'GL Description', 'OA_Difference',
       'Original Budget','Actuals','Revenue or Expenditure']

sort_by = ['GL Account', 'GL Description']

In [82]:
for i, y in enumerate(fy_list):
    print(fy_list[i][essentials].groupby(group_by).sum().shape)
          

(25482, 5)
(25010, 5)
(25320, 5)
(24821, 5)
(27421, 5)
(28308, 5)


# Unique DF - has exact same rows for each year

In [83]:
unique_df = pd.read_csv('unique_df.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [84]:
for i, y in enumerate(fy_list):
    print(y.shape)

(25482, 86)
(25010, 86)
(25320, 86)
(24821, 86)
(27421, 86)
(28308, 86)


In [87]:
unique_df['Fiscal Year'].value_counts()

2015    12310
2014    12310
2013    12310
2012    12310
2011    12310
2010    12310
Name: Fiscal Year, dtype: int64

In [140]:
unique_df.head()

,Unnamed: 0,Actuals,Astros Playoff Rounds,Astros Yearly Attendance,Business Area,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Human Resources Dept.,Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Current Budget,Fiscal Year,Fund Center Id,Fund Id,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Asset Forfeiture Fund - Treasury,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Juvenile Case Manager Fee,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Bldg Security Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,GL Account,GL Category,GL Category Description,GL Description,OA_Difference,Original Budget,Revenue or Expenditure,Rockets Playoff Rounds,Rockets Total Attendance,Texans Average Attendance,Texans Playoff Rounds,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Crime Index,Yearly Rain Fall,key
0,1,960305.36,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1831618.0,2010,1000010001,1000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500010,500,500 - Personnel Services,500010 - Salary Base Pay - Civilian,-37155.64,997461.0,Expenditures,0.0,699280.0,71080.0,0.0,80.0,62.0,596.6,47.01,none
1,2,1258680.84,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1320119.0,2010,1000010001,1000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500020,500,500 - Personnel Services,500020 - Salary Base Pay - Classified,-61438.16,1320119.0,Expenditures,0.0,699280.0,71080.0,0.0,80.0,62.0,596.6,47.01,none
2,4,70887.50,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,75000.0,2010,1000010001,1000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500050,500,500 - Personnel Services,500050 - Salary-Education/Incentive Pay - Classif,-4112.50,75000.0,Expenditures,0.0,699280.0,71080.0,0.0,80.0,62.0,596.6,47.01,none
3,5,12191.97,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1

In [93]:
year_list = [2010, 2011, 2012, 2013, 2014, 2015, 2016]

for index, year in enumerate(fy_list[:6]):
    fy_list[index] = unique_df.loc[unique_df['Fiscal Year']==year_list[index]]

for index, year in enumerate(fy_list[:6]):
    print(index, unique_df.loc[unique_df['Fiscal Year']==year_list[index]].shape)
    print(index, fy_list[index].shape)
    print('')

0 (12310, 87)
0 (12310, 87)

1 (12310, 87)
1 (12310, 87)

2 (12310, 87)
2 (12310, 87)

3 (12310, 87)
3 (12310, 87)

4 (12310, 87)
4 (12310, 87)

5 (12310, 87)
5 (12310, 87)



In [218]:
# for i, y in enumerate(fy_list):
#     for j, yj in enumerate(fy_list):
#         print(fy_list[i][essentials].equals(fy_list[j][essentials]))

In [221]:
hold_df = pd.DataFrame()
for i, y in enumerate(fy_list):
    fy_list[i].index = range(len(y))
    fy_list[i] = y.reset_index(drop=True)
    fy_list[i]=y.sort_values(sort_by)
    hold_df['201'+ str(i)+' GL Account']=fy_list[i]['GL Account']
#     for k in sort_by:
#         print(i,k)
#         hold_df['201'+ str(i)+' '+k]= fy_list[i][k]

hold_df.tail(500)

,2010 GL Account,2011 GL Account,2012 GL Account,2013 GL Account,2014 GL Account,2015 GL Account
11810,522205,522205,522205,522205,522205,522205
11811,522205,522205,522205,522205,522205,522205
11812,522205,522205,522205,522205,522205,522205
11813,522205,522205,522205,522205,522205,522205
11814,522205,522205,522205,522205,522205,522205
11815,522205,522205,522205,522205,522205,522205
11816,522205,522205,522205,522205,522205,522205
11817,522205,522205,522205,522205,522205,522205
11818,522205,522205,522205,522205,522205,522205
11819,522205,522205,522205,522205,522205,522205


In [130]:
# for i, y in enumerate(fy_list[:6]):
#     fy_list[i].to_csv('fy_1'+str(i)+'_uniform'+'.csv')

In [288]:
fy_10 = pd.read_csv('fy_10_uniform.csv')
fy_11 = pd.read_csv('fy_11_uniform.csv')
fy_12 = pd.read_csv('fy_12_uniform.csv')
fy_13 = pd.read_csv('fy_13_uniform.csv')
fy_14 = pd.read_csv('fy_14_uniform.csv')
fy_15 = pd.read_csv('fy_15_uniform.csv')

fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15]

In [289]:
for index, year in enumerate(fy_list[:6]):
    #print(index)
     #if in 2010 lookback zero years
    if index==0:
       
        
        fy_list[index]['3Y_Past_Actuals']="999999999999999"
        fy_list[index]['2Y_Past_Actuals']="999999999999999"
        fy_list[index]['1Y_Past_Actuals']="999999999999999"
        fy_list[index]['2Y_1Y_Actuals_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_Actuals_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['2Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['1Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['2Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['1Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['2Y_1Y_OA_Difference_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_OA_Difference_Mean'] = "999999999999999"
      
    #if in 2011 lookback one year
    elif index==1:
        fy_list[index]['3Y_Past_Actuals']="999999999999999"
        fy_list[index]['2Y_Past_Actuals']="999999999999999"
        fy_list[index]['1Y_Past_Actuals']=fy_list[index-1]['Actuals']
        fy_list[index]['2Y_1Y_Actuals_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_Actuals_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['2Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['1Y_Past_Original_Budget']=fy_list[index-1]['Original Budget']
        fy_list[index]['2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['2Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['1Y_Past_OA_Difference']=fy_list[index-1]['OA_Difference']
        fy_list[index]['2Y_1Y_OA_Difference_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_OA_Difference_Mean'] = "999999999999999"
    
    #if in 2012 lookback two years
    elif index==2:
        fy_list[index]['3Y_Past_Actuals']="999999999999999"
        fy_list[index]['2Y_Past_Actuals']=fy_list[index-2]['Actuals']
        fy_list[index]['1Y_Past_Actuals']=fy_list[index-1]['Actuals']
        hold_df = fy_list[index][["2Y_Past_Actuals","1Y_Past_Actuals"]]
        fy_list[index]['2Y_1Y_Actuals_Mean'] = hold_df.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Actuals_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['2Y_Past_Original_Budget']=fy_list[index-2]['Original Budget']
        fy_list[index]['1Y_Past_Original_Budget']=fy_list[index-1]['Original Budget']
        hold_df = fy_list[index][["2Y_Past_Original_Budget","1Y_Past_Original_Budget"]]
        fy_list[index]['2Y_1Y_Original_Budget_Mean'] = hold_df.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['2Y_Past_OA_Difference']=fy_list[index-2]['OA_Difference']
        fy_list[index]['1Y_Past_OA_Difference']=fy_list[index-1]['OA_Difference']
        hold_df = fy_list[index][["2Y_Past_OA_Difference","1Y_Past_OA_Difference"]]
        fy_list[index]['2Y_1Y_OA_Difference_Mean'] = hold_df.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_OA_Difference_Mean'] = "999999999999999"
        
    #otherwise lookback three years
    else:
        print("else, ", index)
        fy_list[index]['3Y_Past_Actuals']=fy_list[index-3]['Actuals']
        fy_list[index]['2Y_Past_Actuals']=fy_list[index-2]['Actuals']
        fy_list[index]['1Y_Past_Actuals']=fy_list[index-1]['Actuals']
        hold_df_2 = fy_list[index][["2Y_Past_Actuals","1Y_Past_Actuals"]]
        hold_df_3 = fy_list[index][["3Y_Past_Actuals","2Y_Past_Actuals","1Y_Past_Actuals"]]
        fy_list[index]['2Y_1Y_Actuals_Mean'] = hold_df_2.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Actuals_Mean'] = hold_df_2.mean(axis=1)
        
        fy_list[index]['3Y_Past_Original_Budget']=fy_list[index-3]['Original Budget']
        fy_list[index]['2Y_Past_Original_Budget']=fy_list[index-2]['Original Budget']
        fy_list[index]['1Y_Past_Original_Budget']=fy_list[index-1]['Original Budget']
        hold_df_2 = fy_list[index][["2Y_Past_Original_Budget","1Y_Past_Original_Budget"]]
        hold_df_3 = fy_list[index][["3Y_Past_Original_Budget","2Y_Past_Original_Budget","1Y_Past_Original_Budget"]]
        fy_list[index]['2Y_1Y_Original_Budget_Mean'] = hold_df_2.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Original_Budget_Mean'] = hold_df_2.mean(axis=1)
        
        fy_list[index]['3Y_Past_OA_Difference']=fy_list[index-3]['OA_Difference']
        fy_list[index]['2Y_Past_OA_Difference']=fy_list[index-2]['OA_Difference']
        fy_list[index]['1Y_Past_OA_Difference']=fy_list[index-1]['OA_Difference']
        hold_df_2 = fy_list[index][["2Y_Past_OA_Difference","1Y_Past_OA_Difference"]]
        hold_df_3 = fy_list[index][["3Y_Past_OA_Difference","2Y_Past_OA_Difference","1Y_Past_OA_Difference"]]
        fy_list[index]['2Y_1Y_OA_Difference_Mean'] = hold_df_2.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_OA_Difference_Mean'] = hold_df_2.mean(axis=1)
        

else,  3
else,  4
else,  5


In [290]:
fy_list[0].head(1)

,Unnamed: 0,Unnamed: 0.1,Actuals,Astros Playoff Rounds,Astros Yearly Attendance,Business Area,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Human Resources Dept.,Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Current Budget,Fiscal Year,Fund Center Id,Fund Id,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Asset Forfeiture Fund - Treasury,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Juvenile Case Manager Fee,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Bldg Security Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,GL Account,GL Category,GL Category Description,GL Description,OA_Difference,Original Budget,Revenue or Expenditure,Rockets Playoff Rounds,Rockets Total Attendance,Texans Average Attendance,Texans Playoff Rounds,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Crime Index,Yearly Rain Fall,key,3Y_Past_Actuals,2Y_Past_Actuals,1Y_Past_Actuals,2Y_1Y_Actuals_Mean,3Y_2Y_1Y_Actuals_Mean,3Y_Past_Original_Budget,2Y_Past_Original_Budget,1Y_Past_Original_Budget,2Y_1Y_Original_Budget_Mean,3Y_2Y_1Y_Original_Budget_Mean,3Y_Past_OA_Difference,2Y_Past_OA_Difference,1Y_Past_OA_Difference,2Y_1Y_OA_Difference_Mean,3Y_2Y_1Y_OA_Difference_Mean
0,0,1,960305.36,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1831618.0,2010,1000010001,1000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500010,500,500 - Personnel Services,500010 - Salary Base Pay - Civilian,-37155.64,997461.0,Expenditures,0.0,699280.0,71080.0,0.0,80.0,62.0,596.6,47.01,none,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999,999999999999999


In [291]:
categorical_variables = ['GL Category Description']

for i, year in enumerate(fy_list):
    dummies = pd.get_dummies(year[categorical_variables]) # One-hot encoding of categorical variables
    fy_list[i] = pd.concat([year,dummies], axis=1)
    
essentials = [
        'Fiscal Year', 'Fund Center Id', 'Fund Id','GL Account', 'GL Category',
        'GL Description','GL Category Description','Revenue or Expenditure','Current Budget',
        'Original Budget','OA_Difference', 'Actuals','1Y_Past_Actuals','2Y_Past_Actuals','3Y_Past_Actuals',
        '2Y_1Y_Actuals_Mean','3Y_2Y_1Y_Actuals_Mean','3Y_Past_Original_Budget',
        '2Y_Past_Original_Budget','1Y_Past_Original_Budget','2Y_1Y_Original_Budget_Mean',
        '3Y_2Y_1Y_Original_Budget_Mean','3Y_Past_OA_Difference','2Y_Past_OA_Difference',
        '1Y_Past_OA_Difference','2Y_1Y_OA_Difference_Mean','3Y_2Y_1Y_OA_Difference_Mean']

sports = ['Astros Playoff Rounds', 'Astros Yearly Attendance', 
          'Rockets Playoff Rounds','Rockets Total Attendance',
          'Texans Average Attendance',
          'Texans Playoff Rounds',
          'Yearly Avg High Temp',
          'Yearly Avg Low Temp',
          'Yearly Crime Index',
          'Yearly Rain Fall']

other = []
for item in fy_list[1].columns:
    if (item not in essentials) and (item not in sports):
        other.append(item)

for i, y in enumerate(fy_list[:6]):
    hold_df = fy_list[i].copy(deep=True)
    fy_list[i] = hold_df[essentials + sports + other]
    fy_list[i].drop(['key','Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)
    


In [243]:
fy_10_original = pd.read_excel('budget-vs-actuals-revenue-and-expenses-fy10-operating-budget.xlsx')
fy_11_original = pd.read_excel('budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx')
fy_12_original = pd.read_excel('budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx')
fy_13_original = pd.read_excel('budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx')
fy_14_original = pd.read_excel('budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx')
fy_15_original = pd.read_excel('budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx')


In [245]:
fy_10_original.dropna(inplace=True)
fy_11_original.dropna(inplace=True)
fy_12_original.dropna(inplace=True)
fy_13_original.dropna(inplace=True)
fy_14_original.dropna(inplace=True)
fy_15_original.dropna(inplace=True)


In [249]:
fy_10_original.head(1)

,Fiscal Year,Fund Id,Fund Name,Fund Type Descr,Business Area,Business Area Name,Fund Center Id,Fund Center Name,GL Account,GL Description,GL Category,GL Category Description,Commitment Set Description,Revenue or Expenditure,Original Budget,Current Budget,Actuals
0,2010,1000,General Fund,General Funds,1000,Houston Police Department,1000009999,HPD - Unmapped Cost,503040,503040 - Health Insurance - Retiree Classified,500,500 - Personnel Services,50 - Total Expenditures,Expenditures,0.0,0.0,0.0


In [ ]:
fy_original = [fy_10_original,fy_11_original,fy_12_original,fy_13_original,fy_14_original, fy_15_original]



In [250]:
fy_10.head(1)

,Unnamed: 0,Unnamed: 0.1,Actuals,Astros Playoff Rounds,Astros Yearly Attendance,Business Area,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Human Resources Dept.,Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Current Budget,Fiscal Year,Fund Center Id,Fund Id,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Asset Forfeiture Fund - Treasury,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Juvenile Case Manager Fee,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Bldg Security Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,GL Account,GL Category,GL Category Description,GL Description,OA_Difference,Original Budget,Revenue or Expenditure,Rockets Playoff Rounds,Rockets Total Attendance,Texans Average Attendance,Texans Playoff Rounds,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Crime Index,Yearly Rain Fall,key
0,0,1,960305.36,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1831618.0,2010,1000010001,1000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500010,500,500 - Personnel Services,500010 - Salary Base Pay - Civilian,-37155.64,997461.0,Expenditures,0.0,699280.0,71080.0,0.0,80.0,62.0,596.6,47.01,none


In [247]:
fund_id_center_tuples = []
business_area_name_tuples = []

for i, year in enumerate(fy_original):
    business_area_name_items = year.groupby(['Business Area','Business Area Name']).groups.keys()
    fund_id_year_items = year.groupby(['Fund Center Id','Fund Center Name']).groups.keys()
    if i = 0:
        fund_id_center_tuples = list(fund_id_year_items)
        business_area_name_tuples =  list(business_area_name_items)
    else:
        fund_id_center_tuples = list(set(fund_id_year_items) & set(fund_id_center_tuples))
        business_area_name_tuples = list(set(business_area_name_items) & set(business_area_name_tuples))
                                          

[(1000010001, 'HPD-Chief of Police'),
 (1000010002, 'HPD-Budget & Finance'),
 (1000010003, 'HPD-Legal Services'),
 (1000010004, 'HPD-Public Affairs'),
 (1000010005, 'HPD-Public Safety'),
 (1000010006, 'HPD-Chief of Staff'),
 (1000010007, 'HPD-NightCmd/Secur'),
 (1000010008, 'HPD-CrimeAnal-CmdCtr'),
 (1000010010, 'HPD-Strategic Ops'),
 (1000010011, 'HPD-Inspections'),
 (1000010012, 'HPD-Internal Affairs'),
 (1000010013, 'HPD-AlterDisputeReso'),
 (1000010014, 'HPD-Prof Stnds Cmd'),
 (1000010015, 'HPD-Employee Svcs'),
 (1000010016, 'HPD-Training'),
 (1000010017, 'HPD-Cadets'),
 (1000010019, 'HPD-Field Ops'),
 (1000010020, 'HPD-Hmlnd Sec Cmd'),
 (1000010021, 'HPD-North Patrl Cmd'),
 (1000010022, 'HPD-IAH Airport Ptrl'),
 (1000010023, 'HPD-HOU Airport Ptrl'),
 (1000010024, 'HPD-Airport Admin.'),
 (1000010025, 'HPD-Central Patrol'),
 (1000010026, 'HPD-Kingwood Patrol'),
 (1000010027, 'HPD-North Ptrl'),
 (1000010028, 'HPD-Northeast Patrol'),
 (1000010029, 'HPD-Northwest Patrol'),
 (1000010030

In [248]:
gl_category_description_tuples = []


for i, year in enumerate(fy_original):
    gl_category_description_year_items = year.groupby(['Fund Center Id','Fund Center Name']).groups.keys()
    if i = 0:
        gl_category_description_tuples = list(gl_category_description_year_items)
    else:
        gl_category_description_tuples = list(set(gl_category_description_year_items) & set(gl_category_description_tuples))

762

In [294]:
fy_list[2].head()

,Fiscal Year,Fund Center Id,Fund Id,GL Account,GL Category,GL Description,GL Category Description,Revenue or Expenditure,Current Budget,Original Budget,OA_Difference,Actuals,1Y_Past_Actuals,2Y_Past_Actuals,3Y_Past_Actuals,2Y_1Y_Actuals_Mean,3Y_2Y_1Y_Actuals_Mean,3Y_Past_Original_Budget,2Y_Past_Original_Budget,1Y_Past_Original_Budget,2Y_1Y_Original_Budget_Mean,3Y_2Y_1Y_Original_Budget_Mean,3Y_Past_OA_Difference,2Y_Past_OA_Difference,1Y_Past_OA_Difference,2Y_1Y_OA_Difference_Mean,3Y_2Y_1Y_OA_Difference_Mean,Astros Playoff Rounds,Astros Yearly Attendance,Rockets Playoff Rounds,Rockets Total Attendance,Texans Average Attendance,Texans Playoff Rounds,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Crime Index,Yearly Rain Fall,Business Area,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Human Resources Dept.,Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Asset Forfeiture Fund - Treasury,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Juvenile Case Manager Fee,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Bldg Security Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,GL Category Description_500 - Personnel Services,GL Category Description_510 - Supplies,GL Category Description_520 - Other Services and Charges,GL Category Description_530 - Debt Service and Other Uses,GL Category Description_550 - Non-Capital Purchases,GL Category Description_560 - Capital Purchases
0,2012,1000010001,1000,500010,500,500010 - Salary Base Pay - Civilian,500 - Personnel Services,Expenditures,812634.0,1005899.0,-240427.69,765471.31,765846.85,960305.36,999999999999999,863076.105,999999999999999,999999999999999,997461.0,1144554.0,1071007.5,999999999999999,999999999999999,-37155.64,-378707.15,-207931.395,999999999999999,0.0,2067016.0,0.0,663839.0,71665.0,2.0,82.0,63.0,534.2,24.57,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

In [295]:
for i, y in enumerate(fy_list[:6]):
    fy_list[i].to_csv('fy_1'+str(i)+'_augmented'+'.csv')